In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pims import ImageSequence

from lib import luminance
from lib import load

### Video Material

Image data was published as data set at the [VHub](https://vhub.org) platform. Videos of each run are published as separate dataset.
  - `pr06`: https://vhub.org/resources/4211
  - `pr05`: https://vhub.org/resources/4237
  - `ir16`: https://vhub.org/resources/4240
  - `ir15`: https://vhub.org/resources/4246
  - `ir14`: https://vhub.org/resources/4261
  - `ir13`: https://vhub.org/resources/4270
  - `ir12`: https://vhub.org/resources/4279
  - `ir07`: https://vhub.org/resources/4289
  - `ir06`: https://vhub.org/resources/4293
  - `ir05`: https://vhub.org/resources/4299
  - `ir04`: https://vhub.org/resources/4306
  - `ir03`: https://vhub.org/resources/4313

`load.show()` lists the videos that were already analyzed. `load.show(url=True)` will also print the corresponding download urls of each video.

In [ ]:
load.show()

### Create an image sequence from the video file

To ease the download/file naming process, the `load.imseq()` function will test if the video data of a specific experiment (`run`) and camera is already present on the system (i.e. in the `data/` folder). If that is not the case, it will download the video file from VHub, create ana ppropriate folder, and convert it to an image sequence using `ffmpeg` with the following command
```bash
ffmpeg -i data/run_cam.mp4 -q:v 1 data/run_cam/frame%08d.jpg
```
The original video file or zip archives will not be removed after the sequence is created. 
> **note on file and folder names**: We started working with camera names containing the manufacturer, since this is easy to start with. For a publication this is however not optimal. Therefore some of the (later) uploaded video material contains a different camera label. During dowload the `load.ipseq()` function prints out a line containing the file name on disk. Also the [dataset pages on VHub](#Video-material) listed above have the explicit video name convention explicitely listed.

Be aware that some of the video material may be large (>1GB). The `casio-f1` file of the `pr06` run will only use about 200MB disk space:

In [ ]:
seq = load.imgseq(run='pr06', cam='casio-f1')

Once the image sequence is present on the hard disk `load.imseq()` will skip the download and convertion steps and just return the sequence (`pims.ImageSequence`) object. This can be made explicit by replacing the above call with the following line (and un-comment), and adjusting the path specification. The two loading options `as_grey` and `format` should not be changed. They ensure that data is loaded as gray values with an appropriate whit point determined by the image source, and the array's numeryc format is 64-bit float.
```python
seq = ImageSequence('data/pr06_casio-f1/*.jpg', as_grey=True, format=np.float64)
```

In [ ]:
seq[0]

**spatial resolution:**  
Width of container top is 51 cm in real space, 129 px on image frame.

In [ ]:
res = .51 / 129

**Melt brightness:**  
That should be measured from a typical melt 'domain' that shows no sign of motion blur. The measurement tool of ImageJ is the quickes way to do that.

In [ ]:
bmelt = 175.

**Field of view:**  
Visible area is the images resolution multiplied by the images width $W$ and height $H$
$$
S_c = a^2\,W\,H
$$

In [ ]:
H, W = seq[0].shape
Sc = res ** 2 * W * H
Sc

Cumulative melt brightness is the brightness that would be measures if only melt was visible in the image, therefore

In [ ]:
Bmelt = W * H * bmelt

**Background noise:**  
Take all frames before start of water injection to compute the average brightness of that interval. This is the beckground noise. In this video injection starts at frame 94, when the sync light turns on.

In [ ]:
B0 = luminance.average_cbright(seq[:94])

**Luminanace:**  
$$
L = S_c\,\frac{B - B_0}{B_{melt} - B_0}
$$

In [ ]:
L = luminance.luminance_sequence(seq, Sc, Bmelt, B0)

**Time axis:**  
A time array that is 0 at water injection start, with time steps matching those of the video frame rate.

In [ ]:
fps = 300
t = np.linspace(-94 / fps, (len(L) - 94) / fps, len(L))

Plot result

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(t, L)
plt.xlabel(r'$t/\mathrm{s}$')
plt.ylabel(r'$L/\mathrm{m^2}$');

## Optional Functionality
### The `select` switch: Compute $L$ for a selection

 To improve the signal to noise ratio, $L$ can be computed from a subset of each frame. That selection is the same for all frames in the sequence.   
The format is `((vertical start, vertical end), (horizontal start, horizontal end))`

In [ ]:
sel = ((0, 250), (140, 440))

seq[0][0:250, 140:440]

Per pixel values stay unchanges. Cumulative values ($B_{melt}, B_0$) have to be re-computed.

In [ ]:
H1, W1 = 250, 300
Bmelt1 = H1 * W1 * bmelt
Sc1 = H1 * W1 * res ** 2
Sc1

The functions of the `luminance` module have a `select` switch, where appropriate, which use the above format.

In [ ]:
B01 = luminance.average_cbright(seq[:94], select=sel)
L1 = luminance.luminance_sequence(seq, Sc1, Bmelt1, B01, select=sel)

In [ ]:
plt.figure(figsize=(9, 6))
plt.plot(t, L1)
plt.xlabel(r'$t/\mathrm{s}$')
plt.ylabel(r'$L_1/\mathrm{m^2}$');

### Propagation of Uncertainties / Measurement Errors

Image resolutions $a$ (```res``` in the code)
$$
a = \frac{x}{X}\quad,
$$
where $x$ is a known length in real space, measured for example in meters, and $X$ is the same length on the camera's chip, measured in px. In this example the container width is known to be $x=0.51\,\mathrm{m}$ wide, and the accuracy is about $\pm2\,\mathrm{mm}$. In the image sequence the container width is $X=129\,\mathrm{px}$, and the accuracy there is about $\pm1\,\mathrm{px}$.

$$
\sigma_{a}^2 = a^2\,\Bigl(\frac{\sigma_x^2}{x^2} + \frac{\sigma_X^2}{X^2}\Bigr)
$$

In [ ]:
σres = res * np.sqrt((2e-3 / 0.51) ** 2 + (1 / 129.1) ** 2)
res, σres

The camera specific melt brightness of one pixel is ${b_{melt}=175}{}$ (no units here). Playing around with a suitable averaging area shows that this value typically changes by about ${\sigma_{b_{melt}}=\pm5}$. 
$$
\sigma_{B_{melt}} = \sigma_{b_{melt}}\frac{B_{melt}}{b_{melt}}
$$

In [ ]:
σbmelt = 5.
σBmelt = σbmelt * Bmelt / bmelt
σBmelt

Error of $S_c$:

$$
\sigma_{S_c} = 2S_c\,\frac{\sigma_{a}}{a}
$$

In [ ]:
σSc = 2 * Sc * σres / res
σSc

Uncertainty for the background level (noise) $B_0$ comes from the temporal changes during the averaging period (here during the first 94 frames of the image sequence. The `average_cbright()` function has an `uncert` switch. When set to `True` a tuple of $B_0$ and its standard deviation is returned.

In [ ]:
B0, σB0 = luminance.average_cbright(chunk=seq[:94], uncert=True)
B0, σB0

Luminances uncertainty is evaluated as:
$$
\sigma_L^2 = \biggl(\frac{L}{B_{melt}-B_0}\biggr)^2 \sigma_{B_{melt}}^2 +
\biggl(\frac{L - 1}{B_{melt}-B_0}\biggr)^2\sigma_{B_0}^2 +
\biggl(\frac{L}{S_c}\biggr)^2\sigma_{S_c}^2
\quad,
$$
and is available as function `sigma_luminance()`, that returns an array with uncertainties for each frame it was given in the `lum` parameter.

In [ ]:
σL = luminance.sigma_luminance(lum=L, ref=Bmelt, sref=σBmelt, noise=B0, snoise=σB0,
                               fov=Sc, sfov=σSc)

A plot with the standard deviation as gray shadow:

In [ ]:
plt.figure(figsize=(9, 6))
plt.fill_between(t, L1 + σL, L1 - σL, color='#b5b5b5')
plt.plot(t, L1)
plt.xlabel(r'$t/\mathrm{s}$')
plt.ylabel(r'$L_1/\mathrm{m^2}$');